> **Nota edytorska:**
> Dokument opisuje logikę działania modułów `fem_optimizer.py`, `engine_fem.py` oraz `engine_geometry.py` (wersja 4.2). Koncentruje się na metodyce symulacji numerycznej, kryteriach zbieżności oraz przygotowaniu danych do analizy szczegółowej (spoiny).

# 📘 Dokumentacja Techniczna: Analiza Numeryczna i Optymalizacja (FEM)

## 1. Architektura Procesu Numerycznego

W przeciwieństwie do solvera analitycznego (korzystającego z zamkniętych wzorów), moduł FEM wykonuje symulację fizyczną na zdyskretyzowanym modelu 3D. Proces ten jest w pełni zautomatyzowany i sterowany przez klasę `FemOptimizer`.

### Główna pętla sterowania (Optimization Loop)
Algorytm nie wykonuje pojedynczego obliczenia, lecz realizuje iteracyjny proces **$h$-refinementu** (zagęszczania siatki), dążąc do wyniku niezależnego od dyskretyzacji.

1.  **Inicjalizacja:** Pobranie geometrii kandydata (z etapu analitycznego) i ustalenie startowego rozmiaru elementu ($lc_{start}$).
2.  **Generacja Modelu:** Stworzenie geometrii CAD i siatki (Mesh) przy użyciu silnika `Gmsh`.
3.  **Rozwiązanie (Solver):** Uruchomienie solera `CalculiX` (ccx) dla zadanych warunków brzegowych.
4.  **Weryfikacja Zbieżności:** Porównanie maksymalnego naprężenia zredukowanego (Von Mises) z wynikiem poprzedniej iteracji.
    * Jeśli $\Delta < Tolerancja$: Stop, wynik uznany za wiarygodny.
    * Jeśli $\Delta > Tolerancja$: Zmniejszenie rozmiaru siatki o współczynnik $f_{refine}$ (np. $0.7$) i powrót do punktu 2.

---

## 2. Model Geometryczny i Dyskretyzacja

### Topologia (Continuous Mesh)
Model geometryczny tworzony jest jako **jedna bryła wielobjętościowa** (funkcja `fragment` w OpenCascade). 
* Ceowniki i płaskownik są "stopione" ze sobą.
* Na styku elementów (interfejsie) węzły siatki są wspólne (współdzielone).
* **Uzasadnienie:** Eliminuje to konieczność definiowania numerycznie kosztownych kontaktów (`*CONTACT`) lub sztywnych wiązań (`*TIE`), które często generują artefakty naprężeniowe (osobliwości) zaburzające proces optymalizacji.

### Strategia Siatkowania (Meshing)
Jakość siatki jest kluczowa dla wiarygodności wyników. System stosuje:
1.  **Elementy 2. rzędu (C3D10):** Domyślnie używane są tetraedry z węzłami pośrednimi (10 węzłów na element). Pozwalają one na dokładne odwzorowanie pól naprężeń przy zginaniu nawet przy stosunkowo rzadkiej siatce.
2.  **Strefy Zagęszczania (Zones):** Użytkownik może zdefiniować obszary o podwyższonej precyzji (np. `SURF_WELD`, `SURF_FLANGES`). Algorytm aplikuje pola `Box Field` w Gmsh, wymuszając mniejszy rozmiar elementu w newralgicznych miejscach.
3.  **Safety Fuse:** Automatyczna detekcja cienkich ścianek. Jeśli zadany rozmiar siatki jest większy niż grubość najcieńszej blachy, algorytm automatycznie redukuje $lc_{start}$ do wartości grubości ścianki, aby uniknąć elementów zdegenerowanych.

---

## 3. Fizyka i Warunki Brzegowe

Solver `CalculiX` rozwiązuje układ równań równowagi dla ciała stałego deformowalnego. Model uwzględnia:

### Materiał
Właściwości są dynamicznie obliczane na podstawie danych z bazy:
* Moduł Younga ($E$) i Kirchhoffa ($G$) pobierane są z `material_catalogue`.
* **Liczba Poissona ($\nu$):** Obliczana analitycznie z relacji izotropowej: $\nu = \frac{E}{2G} - 1$. Zapewnia to pełną spójność modelu numerycznego z analitycznym.

### Obciążenia i Kinematyka (Rigid Bodies)
Aby uniknąć sztucznych spiętrzeń naprężeń (osobliwości) w punktach przyłożenia sił, zastosowano technikę **Ciał Sztywnych (`*RIGID BODY`)**.
1.  **Podpora ($X=0$):** Wszystkie węzły powierzchni dolnej są powiązane z punktem referencyjnym $RP_{sup}$ (w środku ścinania $S_s$). Warunek utwierdzenia ($U=0, Rot=0$) nakładany jest na ten punkt.
2.  **Obciążenie ($X=L$):** Wszystkie węzły powierzchni górnej powiązane są z $RP_{load}$.
3.  **Mimośród ($M_{ext}$):** Punkt $RP_{load}$ jest przesunięty względem środka ciężkości o ramię działania siły ($Load_Y$). Przyłożenie siły skupionej $F_x$ w tym punkcie automatycznie generuje na przekroju słupa moment gnący $M_z = F_x \cdot e$ oraz moment skręcający (jeśli występuje $F_z$).

### Wektor Sił
Model obsługuje pełny, trójskładowy wektor obciążenia:
* $F_1 (X)$: Osiowe ściskanie.
* $F_2 (Y)$: Ścinanie/zginanie w płaszczyźnie słabej.
* $F_3 (Z)$: Ścinanie/zginanie w płaszczyźnie mocnej + Skręcanie (działanie na ramieniu).

---

## 4. Procedura Rozwiązania (Kroki Solvera)

Plik sterujący `.inp` definiuje sekwencję analiz:

### KROK 1: Statyka Liniowa (`*STATIC`)
Obliczenie stanu równowagi dla zadanych sił. Wynikiem są pola przemieszczeń ($U$) i naprężeń ($S$).
* Wyniki są zapisywane dla całego modelu (`VOL_ALL`) w punktach całkowania.
* Dodatkowo, dla węzłów czujników (`NSET_SENSORS`) i interfejsu (`GRP_INTERFACE`) zapisywane są wartości węzłowe (`*NODE PRINT`), co ułatwia post-processing.

### KROK 2: Wyboczenie (`*BUCKLE`)
Analiza liniowa stateczności (Linear Perturbation). Solver szuka wartości własnych (Eigenvalues) $\lambda$, dla których macierz sztywności staje się osobliwa ($K + \lambda K_G = 0$).
* Wynik: Mnożnik obciążenia krytycznego (Buckling Factor).
* Interpretacja: Jeśli $Factor < 1.0$, konstrukcja ulegnie wyboczeniu przed osiągnięciem pełnego obciążenia roboczego.

---

## 5. System Pomiarowy i Walidacja

Aby umożliwić bezpośrednie porównanie FEM z analityką, system generuje **Wirtualne Sondy (Probes)**.

### Mapowanie Węzłów (`NodeMapper`)
Ponieważ siatka jest nieustrukturyzowana, nie znamy z góry ID węzłów w interesujących nas punktach. Klasa `NodeMapper`:
1.  Wczytuje chmurę punktów z pliku `.nodes.csv`.
2.  Dla każdego zdefiniowanego punktu analitycznego ($P1 \dots P6$) znajduje najbliższy węzeł siatki metodą `Nearest Neighbor`.

### Punkty Użytkownika (Custom Probes)
System pozwala na definiowanie własnych punktów pomiarowych poprzez formuły matematyczne (np. `"tp/2 + hc"`). Są one parsowane w locie i dodawane do mapy czujników. Pozwala to na badanie naprężeń w dowolnym, nietypowym miejscu przekroju bez przebudowy kodu.

### Kryterium Zbieżności
Jako metrykę stopu przyjęto stabilizację globalnego maksimum naprężeń Von Misesa.
$$\delta = \frac{|\sigma_{max}^{i} - \sigma_{max}^{i-1}|}{\sigma_{max}^{i-1}}$$
Jest to kryterium inżynierskie. Należy pamiętać, że w punktach osobliwych (naroża przy utwierdzeniu) naprężenia mogą dążyć do nieskończoności. Dlatego analiza trendu zbieżności jest ważniejsza niż sama wartość bezwzględna w pierwszej iteracji.

---

## 6. Przygotowanie do Analizy Spoin (Interface Analysis)

Moduł FEM przygotowuje dane wejściowe dla przyszłego, szczegółowego modułu wymiarowania spoin.

### Definicja Problemu
Ponieważ model jest ciągły (brak elementów kontaktowych), nie otrzymujemy wprost sił w łącznikach. Naprężenia w spoinie są tożsame z naprężeniami w materiale na styku elementów.

### Metoda Ekstrakcji Danych
1.  **Grupa `GRP_INTERFACE`:** Podczas generowania geometrii, węzły leżące na styku płaskownika i ceownika są automatycznie tagowane.
2.  **Ekstrakcja Tensorów:** Parser wyników (`parse_dat_results`) wyciąga pełny tensor naprężeń ($S_{xx}, S_{yy}, S_{zz}, S_{xy}, S_{xz}, S_{yz}$) dla wszystkich węzłów interfejsu.
3.  **Wstępna ocena:** Obliczana jest maksymalna wartość naprężenia ścinającego w płaszczyźnie spoiny:
    $$\tau_{interface} = \sqrt{S_{xy}^2 + S_{xz}^2}$$
    (Dla interfejsu zorientowanego wzdłuż płaszczyzny X-Z, gdzie Y jest normalną).

### Planowany Algorytm (Next Step)
W kolejnych modułach te dane posłużą do metody **"Wirtualnych Plastrów"**:
* Podział chmury punktów interfejsu na segmenty wzdłuż osi słupa (np. co 50mm).
* Uśrednienie wektorów naprężeń w każdym plastrze.
* Wskazanie segmentu o największym wytężeniu do szczegółowego modelowania spoiny (sub-modeling).